In [1]:
import pandas as pd
import numpy as np
import MDAnalysis as mda

# atom-ID molecule-ID atom-type x y z


In [81]:
u = mda.Universe(
  "/scratch/ws/0/s4610340-bt-eea1-md-workspace/bachelor-thesis/experiments/1-FENE_beadspring/data/FENE_initial_system.data",
  "/scratch/ws/0/s4610340-bt-eea1-md-workspace/bachelor-thesis/experiments/1-FENE_beadspring/data/polymer_relax.out",
  format="LAMMPSDUMP",
  atom_style="id resid type x y z"
)
u

<Universe with 2560 atoms>

In [82]:
u.atoms.groupby("types")

{'3': <AtomGroup with 20 atoms>,
 '1': <AtomGroup with 40 atoms>,
 '2': <AtomGroup with 2500 atoms>}

In [ ]:
u